### **Importation Des Bibliothèques**

In [2]:
import json
import numpy as np
import torch
from transformers import (
    BertTokenizerFast,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from datasets import DatasetDict, Dataset, Features, Sequence, ClassLabel, Value
import evaluate

### **Configuration Des Paramètres Globaux**

In [14]:
MAX_LEN = 135
TRAIN_BATCH_SIZE = 32  # Réduit pour mieux gérer la mémoire GPU
VALID_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 3e-5
BASE_MODEL_PATH = "dslim/bert-base-NER"
MODEL_PATH = "./data/bert-base-NER_model"

# Chemins vers les fichiers de données
TRAINING_FILE = r"D:\AERO_5\Projet atilla\Projet atilla\NER-TRAINING.jsonlines"
VALIDATION_FILE = r"D:\AERO_5\Projet atilla\Projet atilla\NER-VALIDATION.jsonlines"
TESTING_FILE = r"D:\AERO_5\Projet atilla\Projet atilla\NER-TESTING.jsonlines"

# Chemins ou seront stockées les données
TESTING_OUTPUT_FILE = "D:/AERO_5/Projet atilla/Projet atilla/NER-TESTING-PREDICTED.jsonlines"
VALIDATION_OUTPUT_FILE = "D:/AERO_5/Projet atilla/Projet atilla/NER-VALIDATION-PREDICTED.jsonlines"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### **Chargement Du Tokenizer**

In [4]:
TOKENIZER = BertTokenizerFast.from_pretrained(BASE_MODEL_PATH, do_lower_case=True)

# Mapping des labels
ID2LABEL = {
    0: "B-Action",
    1: "B-Entity",
    2: "B-Modifier",
    3: "I-Action",
    4: "I-Entity",
    5: "I-Modifier",
    6: "O"
}
LABEL2ID = {v: k for k, v in ID2LABEL.items()}

### **Chargement Des Données**

In [7]:
# Fonction pour charger un fichier JSONlines
def load_and_prepare_data(file_path):
    """Charge un fichier JSONlines et retourne les données sous forme de liste."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

### **Conversion En Dataset Avec Labels**

In [5]:
def convert_to_dataset_with_labels(data_section, labels):
    """Convertit une section de données en Dataset avec labels dynamiques."""
    has_ner_tags = "ner_tags" in data_section[0]
    
    features = Features({
        "id": Value("int64"),
        "tokens": Sequence(Value("string")),
        **({"ner_tags": Sequence(ClassLabel(names=labels))} if has_ner_tags else {})
    })
    
    dataset_dict = {
        "id": [example["unique_id"] for example in data_section],
        "tokens": [example["tokens"] for example in data_section],
    }
    if has_ner_tags:
        dataset_dict["ner_tags"] = [example["ner_tags"] for example in data_section]
    
    return Dataset.from_dict(dataset_dict, features=features)

### **Chargement Des Données**

In [8]:
# Charger les données
train_data = load_and_prepare_data(TRAINING_FILE)
validation_data = load_and_prepare_data(VALIDATION_FILE)
test_data = load_and_prepare_data(TESTING_FILE)

### **Création De La DataSet**

In [9]:
ner_labels = list(ID2LABEL.values())
ner_data = DatasetDict({
    "train": convert_to_dataset_with_labels(train_data, ner_labels),
    "validation": convert_to_dataset_with_labels(validation_data, ner_labels),
    "test": convert_to_dataset_with_labels(test_data, ner_labels)
})

### **Tokenisation et alignement**

In [10]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """Tokenise les phrases et aligne les étiquettes avec les tokens."""
    tokenized_inputs = TOKENIZER(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Tokenisation des datasets
tokenized_datasets = DatasetDict({
    key: dataset.map(tokenize_and_align_labels, batched=True) if key != "test" else dataset
    for key, dataset in ner_data.items()
})


Map: 100%|██████████| 1044/1044 [00:00<00:00, 12958.29 examples/s]


### **Initialisation du modèle et des arguments**

In [15]:
ner_model = AutoModelForTokenClassification.from_pretrained(
    BASE_MODEL_PATH, 
    num_labels=len(ner_labels), 
    id2label=ID2LABEL, 
    label2id=LABEL2ID,
    ignore_mismatched_sizes=True
)
ner_model.to(DEVICE)

args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01
)

data_collator = DataCollatorForTokenClassification(TOKENIZER)
metric = evaluate.load("seqeval")


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) 

### **Fonction De Calcul Des Métriques**

In [16]:
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds
    pred_logits = np.argmax(pred_logits, axis=2)
    predictions = [[ID2LABEL[p] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(pred_logits, labels)]
    true_labels = [[ID2LABEL[l] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(pred_logits, labels)]
    return metric.compute(predictions=predictions, references=true_labels)

### **Entrainement Du Modèle**

In [17]:
trainer = Trainer(
    model=ner_model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=TOKENIZER,
    compute_metrics=compute_metrics
)

trainer.train()

# Sauvegarde du modèle
ner_model.save_pretrained(MODEL_PATH)
TOKENIZER.save_pretrained(MODEL_PATH)

C:\Users\safah\AppData\Local\Temp\ipykernel_28500\1281199915.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
                                                    
 10%|█         | 153/1530 [13:15<1:22:39,  3.60s/it]

{'eval_loss': 0.37988871335983276, 'eval_Action': {'precision': 0.6014319809069213, 'recall': 0.5009940357852882, 'f1': 0.5466377440347072, 'number': 503}, 'eval_Entity': {'precision': 0.39143426294820716, 'recall': 0.31116389548693585, 'f1': 0.34671371857079836, 'number': 1263}, 'eval_Modifier': {'precision': 0.6284153005464481, 'recall': 0.4107142857142857, 'f1': 0.4967602591792657, 'number': 280}, 'eval_overall_precision': 0.47322540473225405, 'eval_overall_recall': 0.37145650048875856, 'eval_overall_f1': 0.41621029572836804, 'eval_overall_accuracy': 0.8637931560792454, 'eval_runtime': 41.1563, 'eval_samples_per_second': 25.367, 'eval_steps_per_second': 1.604, 'epoch': 1.0}


                                                    
 20%|██        | 306/1530 [29:18<1:40:28,  4.93s/it]

{'eval_loss': 0.3448692262172699, 'eval_Action': {'precision': 0.6270783847980997, 'recall': 0.5248508946322068, 'f1': 0.5714285714285715, 'number': 503}, 'eval_Entity': {'precision': 0.44842562432138977, 'recall': 0.3269992082343626, 'f1': 0.3782051282051282, 'number': 1263}, 'eval_Modifier': {'precision': 0.6948051948051948, 'recall': 0.3821428571428571, 'f1': 0.4930875576036866, 'number': 280}, 'eval_overall_precision': 0.5240641711229946, 'eval_overall_recall': 0.3831867057673509, 'eval_overall_f1': 0.4426877470355731, 'eval_overall_accuracy': 0.8698983485454379, 'eval_runtime': 54.5619, 'eval_samples_per_second': 19.134, 'eval_steps_per_second': 1.21, 'epoch': 2.0}


                                                         
 30%|███       | 459/1530 [1:12:44<1:46:05,  5.94s/it]

...

 98%|█████████▊| 1500/1530 [3:11:32<02:18,  4.60s/it]

{'loss': 0.0231, 'grad_norm': 0.7719393968582153, 'learning_rate': 5.882352941176471e-07, 'epoch': 9.8}


                                                     
100%|██████████| 1530/1530 [3:14:55<00:00,  7.64s/it]


{'eval_loss': 0.5903509855270386, 'eval_Action': {'precision': 0.6384479717813051, 'recall': 0.7196819085487077, 'f1': 0.6766355140186916, 'number': 503}, 'eval_Entity': {'precision': 0.5459811730629979, 'recall': 0.5969912905779889, 'f1': 0.5703479576399395, 'number': 1263}, 'eval_Modifier': {'precision': 0.6492537313432836, 'recall': 0.6214285714285714, 'f1': 0.635036496350365, 'number': 280}, 'eval_overall_precision': 0.5821299638989169, 'eval_overall_recall': 0.6304985337243402, 'eval_overall_f1': 0.6053496011262318, 'eval_overall_accuracy': 0.8916023077627522, 'eval_runtime': 36.5057, 'eval_samples_per_second': 28.598, 'eval_steps_per_second': 1.808, 'epoch': 10.0}
{'train_runtime': 11695.7968, 'train_samples_per_second': 4.169, 'train_steps_per_second': 0.131, 'train_loss': 0.13820040559067445, 'epoch': 10.0}


('./data/bert-base-NER_model\\tokenizer_config.json',
 './data/bert-base-NER_model\\special_tokens_map.json',
 './data/bert-base-NER_model\\vocab.txt',
 './data/bert-base-NER_model\\added_tokens.json',
 './data/bert-base-NER_model\\tokenizer.json')

### **Prédictions**

In [18]:
def predict_and_save(dataset, output_file):
    """Prédit les étiquettes pour le dataset et les sauvegarde dans un fichier."""
    predictions = []
    for example in dataset:
        tokens = example["tokens"]
        inputs = TOKENIZER(tokens, truncation=True, is_split_into_words=True, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            outputs = ner_model(**inputs)
        logits = outputs.logits
        predicted_ids = torch.argmax(logits, dim=-1).squeeze().tolist()
        predicted_labels = [ID2LABEL[i] for i in predicted_ids]
        predictions.append({"tokens": tokens, "predictions": predicted_labels})
    
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(predictions, f, indent=4)

predict_and_save(ner_data["test"], TESTING_OUTPUT_FILE)
predict_and_save(ner_data["validation"], VALIDATION_OUTPUT_FILE)

print("Prédictions sauvegardées avec succès.")

Prédictions sauvegardées avec succès.
